In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [31]:
# Load the IMDb dataset
max_features = 10000    # Vocabulary Size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((25000,), (25000,), (25000,), (25000,))

In [32]:
# Inspect a sample review and its label
sample_review=X_train[0]
sample_label = y_train[0]

In [33]:
# Mapping index back to words (for self understanding) 
word_index = {v:k for k,v in imdb.get_word_index().items()}
sample_review_in_text = " ".join([word_index.get(word-3, "?") for word in sample_review])
sample_review_in_text

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [34]:
max_len = max([len(l) for l in X_train])
max_len

2494

In [35]:
max_len = 500

In [36]:
# Pad Sequences
X_train = sequence.pad_sequences(X_train, maxlen=max_len, padding='pre')
X_test = sequence.pad_sequences(X_test, maxlen=max_len, padding='pre')

In [37]:
# Train simple RNN
model = Sequential()
model.add(Embedding(max_features, 128, input_length = max_len))
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

c:\ProgramData\anaconda3\envs\NLP_ENV\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [38]:
# Create early stopping callback
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
es_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tf_callback = TensorBoard(log_dir='logs/simple_rnn_fit/', histogram_freq=1)

In [39]:
# Fit the model
history = model.fit(
    X_train, y_train, validation_split=0.2,
    epochs=10, batch_size=32, callbacks=[es_callback]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 65ms/step - accuracy: 0.5913 - loss: 1.3006 - val_accuracy: 0.7552 - val_loss: 0.5022
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.8275 - loss: 0.3943 - val_accuracy: 0.8026 - val_loss: 0.4173
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.8879 - loss: 0.2859 - val_accuracy: 0.8486 - val_loss: 0.3946
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 65ms/step - accuracy: 0.9205 - loss: 0.2037 - val_accuracy: 0.8226 - val_loss: 0.4161
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 66ms/step - accuracy: 0.9418 - loss: 0.1655 - val_accuracy: 0.8236 - val_loss: 0.4649
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.9404 - loss: 0.1627 - val_accuracy: 0.8368 - val_loss: 0.4812
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.9684 - loss: 0.0928 - val_accuracy: 0.8202 - val_loss: 0.5547
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 68ms/step - accuracy: 0.9715 - loss: 0.0814 - 

In [42]:
# Save the model
model.save('simple_rnn.h5')